<a href="https://colab.research.google.com/github/gabrielcerono/GlioblastomaMultiforme/blob/main/Survival_Analysis_Lammer2021.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade pip
!pip uninstall --yes --quiet osqp
!pip install -U scikit-survival

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sklearn as sk

In [3]:
!pip install scipy --upgrade

  Using cached scipy-1.7.0-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.whl (28.5 MB)
  Attempting uninstall: scipy
    Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1:
      Successfully uninstalled scipy-1.4.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [2]:
from scipy.stats import rankdata


In [3]:
pip install eli5

     |████████████████████████████████| 106 kB 38.8 MB/s 


In [4]:
from sksurv.linear_model import CoxPHSurvivalAnalysis
from sksurv.metrics import concordance_index_censored
from sksurv.metrics import brier_score

In [5]:
dataset = pd.read_excel('/content/glioma.xlsx')

In [6]:
dataset.head()

,patient ID,sex,age [years],OS [months],"OS status (1=deceased, 0=living)",PFS [months],"progression (1=yes, 0=no)","MGMT methylation (1=methylated, 0=unmethylated)",cHsp70
0,1,m,62,1.2,1,1.2,1,0,high
1,2,m,58,20.9,1,18.2,1,0,high
2,3,f,50,4.0,1,4.6,1,0,low
3,4,m,20,26.0,1,16.1,1,1,high
4,6,m,42,15.8,1,6.2,1,0,low


In [7]:
dataset = dataset.drop(columns=['patient ID'])

In [8]:
dataset.columns

Index(['sex', 'age [years]', 'OS [months]', 'OS status (1=deceased, 0=living)',
       'PFS [months]', 'progression (1=yes, 0=no)',
       'MGMT methylation (1=methylated, 0=unmethylated)', 'cHsp70'],
      dtype='object')

In [9]:
dataset = dataset.rename(columns={'age [years]' : 'age', 'OS [months]' : 'os', 'OS status (1=deceased, 0=living)' : 'death',
       'progression (1=yes, 0=no)' : "didprogress",'MGMT methylation (1=methylated, 0=unmethylated)' : 'mgmt_methylation', 'cHsp70' : 'chsp70_0h1l', 'PFS [months]': 'pfs'})

In [10]:
dataset.head()

,sex,age,os,death,pfs,didprogress,mgmt_methylation,chsp70_0h1l
0,m,62,1.2,1,1.2,1,0,high
1,m,58,20.9,1,18.2,1,0,high
2,f,50,4.0,1,4.6,1,0,low
3,m,20,26.0,1,16.1,1,1,high
4,m,42,15.8,1,6.2,1,0,low


In [11]:
from sklearn.preprocessing import LabelEncoder
from pandas import get_dummies

In [12]:
label_encoder = LabelEncoder()
dataset['sex'] = label_encoder.fit_transform(dataset['sex'])
dataset['chsp70_0h1l'] = label_encoder.fit_transform(dataset['chsp70_0h1l'])

Let's try to predict first overall survival

1.   We need to map death as True or False
2.   We need to put death event first, then the right censored data. Example : (True, 21 months) 
3.   Then we have to pass from pandas DataFrame to structured array
4.   We can recover the data frame from records




In [13]:
dataset.corr(method='pearson')

,sex,age,os,death,pfs,didprogress,mgmt_methylation,chsp70_0h1l
sex,1.000000,-0.269165,-0.016478,0.138013,0.039569,-0.010193,-0.168035,-0.058461
age,-0.269165,1.000000,0.061198,-0.080623,0.177742,-0.161440,0.118375,-0.079750
os,-0.016478,0.061198,1.000000,-0.185224,0.873120,-0.288235,0.330128,-0.313911
death,0.138013,-0.080623,-0.185224,1.000000,-0.223676,0.369274,-0.265897,0.127076
pfs,0.039569,0.177742,0.873120,-0.223676,1.000000,-0.456801,0.320183,-0.328388
didprogress,-0.010193,-0.161440,-0.288235,0.369274,-0.456801,1.000000,-0.010336,0.229416
mgmt_methylation,-0.168035,0.118375,0.330128,-0.265897,0.320183,-0.010336,1.000000,-0.386500
chsp70_0h1l,-0.058461,-0.079750,-0.313911,0.127076,-0.328388,0.229416,-0.386500,1.000000


In [14]:
Y = dataset[['death', 'os']]

In [15]:
Y['death'] = Y['death'].map({1: True, 0: False})

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [16]:
Y = Y.to_records(index=False)

In [17]:
X = dataset.drop(columns = ['death', 'os'])

In [18]:
from sksurv.ensemble import RandomSurvivalForest
from sklearn.model_selection import train_test_split


In [19]:
rsf = RandomSurvivalForest()

In [20]:
from eli5.sklearn import PermutationImportance
import eli5


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)


In [21]:
from sksurv.metrics import (concordance_index_censored,
                            concordance_index_ipcw,
                            cumulative_dynamic_auc)

Let's build a function, to use a rank feature to permutate, and select feature importance and a test split to check for the C-score


In [22]:
def permutation(X, y, model, loops):
  rankings = np.zeros(len(X.columns),)
  bordarank = np.zeros(len(X.columns),)
  c_score_total = []
  a= 0
  np.random.seed(42); randseed = np.random.randint(9999, size = 1500)

  for x in range(loops):

    #Let's do this trick with the seeding, so we can have the same test set
    a += 1
    seed = randseed[a]
  #Let's do a splitting 1/3 train, 1/3 rank, 1/3 test
    X_train, X_test, y_train, y_test = train_test_split(
    X, Y, test_size=0.33, random_state = seed )
    X_train, X_rank, y_train, y_rank = train_test_split(
    X_train, y_train, test_size=0.5, random_state = seed)
  #Let's train the model on the 
    model.fit(X_train, y_train)
  #Let's define the permutation.
    permuter = PermutationImportance(
    estimator = model,
    n_iter = 10)
  #Let's fit the permutator on the Y test, (This permutation only shuffle the Y test.)
    permuter.fit(X_rank, y_rank)
  
    columns = X_test.columns.to_list()
  #The feature importance will give us and n array of how much the Concordance score dropped
    feature_importance = permuter.feature_importances_
  # I need to create a ranking out of this
    rankings = np.add(feature_importance, rankings)
  #This will create a numpy array with each c index loss. Each row is a loop. 
    bordarank = np.vstack((bordarank, feature_importance))
  #Let's compute the Concordance index score
    c_score = model.score(X_test, y_test)
    c_score_total.append(c_score)

  
    
  #Dividing to get the average 
  ranking_terminal = np.true_divide(rankings, loops) 
  c_rank = pd.DataFrame({'features': columns, 'C_loss': ranking_terminal}).sort_values(ascending = False, by =['C_loss'])
  c_mean = np.mean(np.array(c_score_total))
  c_std = np.std(np.array(c_score_total))
  #Let's start working with the borda. First we delete the first row that is just 0s
  bordarank = np.delete(bordarank, 0, 0)
  ranking_the_data = rankdata(bordarank * -1, axis=1)
  rankavg = np.mean(ranking_the_data, axis = 0)
  ranksd = np.std(ranking_the_data, axis = 0)
  #Vamos a ver como queda
  borda = pd.DataFrame({'Features': columns, 'C_Average_Loss': ranking_terminal, 'borda_average': rankavg, 'borda_sd': ranksd}).sort_values(ascending = True, by = ['borda_average'])
  



  return c_rank, c_mean, c_std, borda

Let's try a 1000 looping permutation 

In [23]:
rankfinal, c_score, c_std, borda = permutation(X, Y, rsf, 100)

In [24]:
c_score

0.7321947980182856

In [25]:
c_std

0.074119700178619

In [26]:
borda

,Features,C_Average_Loss,borda_average,borda_sd
2,pfs,0.185322,1.11,0.598247
4,mgmt_methylation,0.018294,3.43,1.423060
5,chsp70_0h1l,0.009145,3.62,1.433736
3,didprogress,0.000690,4.10,1.236932
1,age,-0.002259,4.36,1.493452
0,sex,-0.005237,4.38,1.302152


Let's run again, but with the top 2 features using 

In [ ]:
X_tf = dataset[['pfs', 'mgmt_methylation']]

In [ ]:
def top2features(X, y, model, loops):
  
  c_score_total = []
  a= 0
  np.random.seed(42); randseed = np.random.randint(9999, size = 1500)
  for x in range(loops):

    #Let's do this trick with the seeding, so we can have the same array of test sets, that we had in the previous feature selection algorithm. 
    a += 1
    seed = randseed[a]
  #Let's do a splitting 1/3 train, 1/3 rank, 1/3 test
    X_train, X_test, y_train, y_test = train_test_split(
    X, Y, test_size=0.33, random_state = seed )
    X_train, X_rank, y_train, y_rank = train_test_split(
    X, Y, test_size=0.5, random_state = seed)
  #Let's train the model on the 
    model.fit(X_train, y_train)
  #Let's compute the scores
    c_score = model.score(X_test, y_test)
    c_score_total.append(c_score)

  
  c_mean_top2 = np.mean(np.array(c_score_total))
  c_std = np.std(np.array(c_score_total))



  return c_mean_top2, c_std

In [ ]:
c_score_tf, c_std_tf = top2features(X_tf, Y, rsf, 100)

Prediction using only the top 2 features


In [ ]:
c_score_tf

0.7704624920499491

Pysurvival to get the brier score 

In [27]:
pip install pysurvival

     |████████████████████████████████| 4.7 MB 36.0 MB/s 
  Created wheel for pysurvival: filename=pysurvival-0.1.2-cp37-cp37m-linux_x86_64.whl size=3773277 sha256=2d805e0f4975ec328ba888afd09dcb3b96164c791927301c0aa7fdbd2adf7b16
  Stored in directory: /root/.cache/pip/wheels/1a/63/e2/32273d765a4e2f4ccac69c8adf97425ca80bab5d0c8447f120
  Created wheel for progressbar: filename=progressbar-2.5-py3-none-any.whl size=12082 sha256=3b6ea0cfbeb7780b75afa4e4a46d0e733679c05ac265dccdb3a2ffc2571a1523
  Stored in directory: /root/.cache/pip/wheels/f0/fd/1f/3e35ed57e94cd8ced38dd46771f1f0f94f65fec548659ed855
Successfully built pysurvival progressbar


In [28]:
from pysurvival.utils.metrics import concordance_index

In [29]:
from pysurvival.utils.metrics import integrated_brier_score


In [30]:
from pysurvival.models.survival_forest import RandomSurvivalForestModel

In [31]:
pyforest = RandomSurvivalForestModel(num_trees=100)

x = covariates
t = times
e = event

In [32]:
T = dataset['os']
E = dataset['death']

In [33]:
X_train, X_test, t_train, t_test, e_train, e_test = train_test_split(
    X, T, E, test_size=0.33)

In [34]:
pyforest.fit(X_train, t_train, e_train, max_depth=5)


RandomSurvivalForestModel

In [35]:
a = concordance_index(pyforest, X_test, t_test, e_test)

In [36]:
def Py_scores(X, T, E, model, loops):
  
  c_score_total = []
  ibstotal = []
  a= 0
  np.random.seed(42); randseed = np.random.randint(9999, size = 1500)
  for x in range(loops):

    #Let's do this trick with the seeding, so we can have the same array of test sets, that we had in the previous feature selection algorithm. 
    a += 1
    seed = randseed[a]
  #Let's do a splitting 1/3 train, 1/3 rank, 1/3 test
    X_train, X_test, t_train, t_test, e_train, e_test = train_test_split(
    X, T, E, test_size=0.33, random_state = seed )
    X_train, X_rank, t_train, t_rank, e_train, e_rank = train_test_split(
    X_train, t_train, e_train, test_size=0.5, random_state = seed)
  #Let's train the model on the 
    model.fit(X_train, t_train, e_train, max_depth=5)
    
  #Let's compute the C - scores
  #Let's compute the integrated brier score
    ibs = integrated_brier_score(model, X_test, t_test, e_test)
    ibstotal.append(ibs)
    
  
  
  ibstotal2 = np.mean(np.array(ibstotal))
  ibs_std = np.std(np.array(ibstotal))


  return ibstotal2, ibs_std

In [37]:
ibs_mean, ibs_std = Py_scores(X, T, E, pyforest, 100)

/usr/local/lib/python3.7/dist-packages/pysurvival/models/survival_forest.py:399: RuntimeWarning: invalid value encountered in true_divide
  self.variable_importance_table['pct_importance']= importance/sum_imp


In [38]:
ibs_mean

0.18456687737515107

In [39]:
ibs_std

0.02605649956105561

##Deep Surv

In [40]:
from pysurvival.models.semi_parametric import NonLinearCoxPHModel


In [41]:
def Py_scores_deepsurv(X, T, E, loops):
  
  c_score_total = []
  ibstotal = []
  a= 0
  np.random.seed(42); randseed = np.random.randint(9999, size = 1500)
  for x in range(loops):

    #Let's do this trick with the seeding, so we can have the same array of test sets, that we had in the previous feature selection algorithm. 
    a += 1
    seed = randseed[a]
  #Let's do a splitting 1/3 train, 1/3 rank, 1/3 test
    X_train, X_test, t_train, t_test, e_train, e_test = train_test_split(
    X, T, E, test_size=0.33, random_state = seed )
    X_train, X_rank, t_train, t_rank, e_train, e_rank = train_test_split(
    X_train, t_train, e_train, test_size=0.5, random_state = seed)
  #Let's train the NN 
    structure = [ {'activation': 'ReLU', 'num_units': 128}, {'activation': 'ReLU', 'num_units': 64}, ]
    nonlinear_coxph = NonLinearCoxPHModel(structure = structure)
    nonlinear_coxph.fit(X_train, t_train, e_train, num_epochs = 100, dropout = 0.2,)
    c_score_total.append(concordance_index(nonlinear_coxph, X_test, t_test, e_test))
    
  #Let's compute the C - scores
  #Let's compute the integrated brier score
    ibs = integrated_brier_score(nonlinear_coxph, X_test, t_test, e_test)
    ibstotal.append(ibs)
    
  
  
  ibstotal2 = np.mean(np.array(ibstotal))
  ibs_std = np.std(np.array(ibstotal))
  c_score_avg = np.mean(np.array(c_score_total))
  c_score_std = np.std(np.array(c_score_total))

  return ibstotal2, ibs_std, c_score_avg, c_score_std

In [42]:
ib, ibstd, c_index_avg, c_score_std = Py_scores_deepsurv(X, T, E, 100)

C score average

In [43]:
c_index_avg

0.7213902359108088

C score std

In [44]:
c_score_std

0.08515204737309023

Brier integrated score

In [45]:
ib

0.14598840132474655

Brier std 

In [46]:
ibstd

0.02265937267856033

## Standard Cox Proportional Hazards

In [47]:
from pysurvival.models.semi_parametric import CoxPHModel
  

In [48]:
coxph = CoxPHModel()


In [57]:
def Py_scores_cox(X, T, E, model, loops):
  
  c_score_total = []
  ibstotal = []
  a= 0
  np.random.seed(42); randseed = np.random.randint(9999, size = 1500)
  for x in range(loops):

    #Let's do this trick with the seeding, so we can have the same array of test sets, that we had in the previous feature selection algorithm. 
    a += 1
    seed = randseed[a]
  #Let's do a splitting 1/3 train, 1/3 rank, 1/3 test
    X_train, X_test, t_train, t_test, e_train, e_test = train_test_split(
    X, T, E, test_size=0.33, random_state = seed )
    X_train, X_rank, t_train, t_rank, e_train, e_rank = train_test_split(
    X_train, t_train, e_train, test_size=0.5, random_state = seed)
  #Let's train the model on the 
    model.fit(X_train, t_train, e_train)
    c_score_total.append(concordance_index(model, X_test, t_test, e_test))
    
  #Let's compute the C - scores
  #Let's compute the integrated brier score
    ibs = integrated_brier_score(model, X_test, t_test, e_test)
    ibstotal.append(ibs)
    
  
  
  ibstotal2 = np.mean(np.array(ibstotal))
  ibs_std = np.std(np.array(ibstotal))
  c_index = np.mean(np.array(c_score_total))
  c_index_std = np.std(np.array(c_score_total))

  return ibstotal2, ibs_std, c_index, c_index_std

In [ ]:
ib, ibstd, c_index_avg, c_index_std = Py_scores_cox(X, T, E, coxph, 100)

In [59]:
ib

0.13947753114296663

In [60]:
ibstd

0.03964169629823249

In [61]:
c_index_avg

0.716389208095312

In [62]:
c_index_std

0.10083234357151273